In [ ]:
import itertools
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.utils import shuffle
from sklearn.feature_selection import mutual_info_classif as MIF
import random
import os
# chain()可以把一组迭代对象串联起来，形成一个更大的迭代器
from itertools import chain
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import optimizers, layers, losses, metrics
from tensorflow.keras.initializers import glorot_normal, he_normal
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Embedding, Lambda, multiply, Flatten, Concatenate
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2

from deepctr.inputs import get_dense_input, create_embedding_matrix, embedding_lookup, get_dense_input, varlen_embedding_lookup, \
    get_varlen_pooling_list, mergeDict
from deepctr.layers.sequence import SequencePoolingLayer, Transformer, AttentionSequencePoolingLayer

from deepctr.feature_column import  SparseFeat, DenseFeat, VarLenSparseFeat, get_feature_names, build_input_features, get_linear_logit, DEFAULT_GROUP_NAME, input_from_feature_columns
from deepctr.layers.core import PredictionLayer, DNN
from deepctr.layers.interaction import FM, FEFMLayer, BiInteractionPooling, AFMLayer, CIN, InteractingLayer, FwFMLayer, InnerProductLayer, OutterProductLayer, FGCNNLayer, CrossNet,  CrossNetMix
from deepctr.layers.utils import concat_func, add_func, Hash, NoMask, combined_dnn_input, reduce_sum, softmax
from tensorflow.keras.layers import Layer
from tensorflow.keras.initializers import Zeros, glorot_normal, RandomNormal
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
def NFMDeepFEFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 128, 64),
        l2_reg_embedding=1e-5, l2_reg_linear=1e-5, l2_reg_embedding_feat=0.00001,
             l2_reg_embedding_field=0.00001,l2_reg_dnn=0, seed=1024, bi_dropout=0,
        dnn_dropout=0, dnn_activation='relu', task='binary'):

    features = build_input_features(
        linear_feature_columns + dnn_feature_columns)

    inputs_list = list(features.values())

    linear_logit = get_linear_logit(features, linear_feature_columns, seed=seed, prefix='linear',
                                    l2_reg=l2_reg_linear)

#     sparse_embedding_list, dense_value_list = input_from_feature_columns(features, dnn_feature_columns,
#                                                                          l2_reg_embedding, seed)


    group_embedding_dict, dense_value_list = input_from_feature_columns(features, dnn_feature_columns,
                                                                        l2_reg_embedding_feat,
                                                                        seed, support_group=True)
    sparse_embedding_list = list(chain.from_iterable(group_embedding_dict.values()))
    fm_input = concat_func(sparse_embedding_list, axis=1)

    fefm_interaction_embedding = concat_func([FEFMLayer(
        regularizer=l2_reg_embedding_field)(concat_func(v, axis=1))
                                              for k, v in group_embedding_dict.items() if k in [DEFAULT_GROUP_NAME]],axis=1)


    fefm_logit = tf.keras.layers.Lambda(lambda x: reduce_sum(x, axis=1, keep_dims=True))(fefm_interaction_embedding)
    bi_out = BiInteractionPooling()(fm_input)
    if bi_dropout:
        bi_out = tf.keras.layers.Dropout(bi_dropout)(bi_out, training=None)
    dnn_input = combined_dnn_input([fefm_logit], dense_value_list)
    dnn_output = DNN(dnn_hidden_units, dnn_activation, l2_reg_dnn, dnn_dropout, False, seed=seed)(dnn_input)
    dnn_logit = tf.keras.layers.Dense(
        1, use_bias=False, kernel_initializer=tf.keras.initializers.glorot_normal(seed))(dnn_output)

    final_logit = add_func([linear_logit, dnn_logit])

    output = PredictionLayer(task)(final_logit)

    model = tf.keras.models.Model(inputs=inputs_list, outputs=output)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), 
               loss=losses.BinaryCrossentropy(), 
               metrics=['AUC', 'binary_accuracy', 'Precision', 'Recall'])
    return model

In [ ]:
metrics = []
lrs = [1]
for i in lrs:
    metric = []
    for j in range(1):
        model = NFMDeepFEFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 128, 64),
        l2_reg_embedding=1e-5, l2_reg_linear=1e-5, l2_reg_embedding_feat=0.00001,
             l2_reg_embedding_field=0.00001,l2_reg_dnn=0, seed=1024, bi_dropout=0,
        dnn_dropout=0, dnn_activation='relu', task='binary')
        input_train = deepfm_train
        model.fit(input_train,
                        label,
                        validation_split=0.2,
                        epochs=100,
                        batch_size=128,
                        shuffle = False,
                        verbose = 1, 
                        callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],)
        input_test = deepfm_test
        ans_mtx = model.predict(input_test, batch_size=100)
        loss, auc, acc, pre, rec = model.evaluate(input_test, label1)
        metric.append([loss, auc, acc, pre, rec])
    metric = np.array(metric)
    print(metric)
    metrics.append(metric)
print(metrics)

In [ ]:
print(metrics)
print(*zip(lrs, np.mean(metrics, axis = 1)))